In [6]:

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
with open('/content/drive/MyDrive/TinyStories-valid.txt', 'r', encoding='utf-8') as f:
   text=f.read()

print(len(text))


19432979


In [8]:
# -*- coding: utf-8 -*-
"""tinygptforcollab

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/19DiOjHtL-UEa69WlOcMsmRnkrNc5NmaS
"""

# -*- coding: utf-8 -*-
"""tinygpt.ipynb
Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/109yvdricQdl9D43KPCeh829Y89TGUcvC


"""
#hypparameters
import torch
device='cuda' if torch.cuda.is_available() else 'cpu'
eval_iters=200
n_embd=384
lr=3e-4
batch_size=64
block_size=256
num_heads=6
eval_inter=500
dropout=0.2
nlayerb=6
max_iters=5000

chars=sorted(list((set(text))))
vocab_size=len(chars)

itos={i:s for i, s in enumerate(chars)}
stoi={s:i for i, s in enumerate(chars)}

encode = lambda e: [stoi[c] for c in e]
decode = lambda d: "".join([itos[c] for c in d])




data=torch.tensor(encode(text))
n=int(0.9*len(data))
train_data=data[:n]
val_data=data[n:]

torch.manual_seed(1337)

def get_batch(split):
    data = train_data if split=='train' else val_data
    ix=torch.randint(len(data)-block_size, (batch_size, ))
    x=torch.stack([data[i:i+block_size]for i in ix])
    y=torch.stack([data[i+1:i+block_size+1]for i in ix])
    return x, y

xb, yb=get_batch('train')

import torch
from torch import nn as nn
import torch.nn.functional as F

class Head(nn.Module):
   def __init__(self, head_size):
    super().__init__()
    self.key=nn.Linear(n_embd, head_size, bias=False)
    self.query=nn.Linear(n_embd, head_size, bias=False)
    self.value=nn.Linear(n_embd, head_size, bias=False)
    self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
    self.ddp=nn.Dropout(dropout)
   def forward(self, x):
      B, T, C = x.shape
      k=self.key(x)
      q=self.query(x)
      v=self.value(x)
      wei=q@k.transpose(-2, -1) * k.shape[-1]**-0.5
      wei=wei.masked_fill(self.tril[:T, :T]==0, float('-inf'))
      wei=F.softmax(wei, dim=-1)
      wei=self.ddp(wei)
      if torch.isnan(wei).any() or torch.isinf(wei).any():
          print("NaN or Inf values detected in the probability tensor.")
    # Add debugging information or further investigation her
      else:
         out=wei@v
         return out

class MultiHeadAttention(nn.Module):
   def __init__(self, num_heads, head_size):
      super().__init__()
      self.heads=nn.ModuleList([Head(head_size) for _ in range(num_heads)])
      self.proj=nn.Linear(n_embd, n_embd)
      self.ddp=nn.Dropout(dropout)
   def forward(self, x):
      out=torch.cat([h(x) for h in self.heads], dim=-1)
      out=self.ddp(self.proj(out))
      return out


class feedfoward(nn.Module):
  def __init__(self, n_embd):
    super().__init__()
    self.net=nn.Sequential(nn.Linear(n_embd, 4*n_embd), nn.ReLU(), nn.Linear(4*n_embd, n_embd), nn.Dropout(dropout))
  def forward(self, x):
    ffd=self.net(x)
    return ffd

class Block(nn.Module):
  def __init__(self, n_embd, num_heads):
    super().__init__()
    head_size=n_embd//num_heads
    self.heads=MultiHeadAttention(num_heads, head_size)
    self.ffd=feedfoward(n_embd)
    self.lln=nn.LayerNorm(n_embd)
  def forward(self, x):
    x=x+self.heads(self.lln(x))
    x=x+self.ffd(self.lln(x))
    return x







torch.manual_seed(1337)
class BigramModel(nn.Module):
    def __init__ (self):
        super().__init__()
        self.tokenembeddingtable=nn.Embedding(vocab_size, n_embd)
        self.positionembeddingtable=nn.Embedding(block_size, n_embd)
        self.heads=MultiHeadAttention(num_heads, n_embd//4)
        self.ffd=feedfoward(n_embd)
        self.block=nn.Sequential(*[Block(n_embd, num_heads=num_heads)for _ in range(nlayerb)])
        self.lnf=nn.LayerNorm(n_embd)
        self.lmhead=nn.Linear(n_embd, vocab_size)
    def forward(self, idx, targets=None):
        idx = idx.to(device)
        B, T= idx.shape
        tok_emb=self.tokenembeddingtable(idx)
        pos_emb=self.positionembeddingtable(torch.arange(T, device=device))
        x=tok_emb+pos_emb
        x=self.block(x)
        x=self.lnf(x)

        logits=self.lmhead(x)


        B, T, C =logits.shape

        if targets is not None:
          logits=logits.view(B*T, C)
          targets=targets.to(device)

          targets=targets.view(B*T)
          loss=F.cross_entropy(logits, targets)
        else:
          loss=None
        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
          idx_cong=idx[:, -block_size:]
          logits, loss=self(idx_cong)
          logits=logits[:, -1, :]
          prob=F.softmax(logits, dim=-1)
          idx_next=torch.multinomial(prob, num_samples=1)
          idx=torch.cat((idx, idx_next), dim=1)


        return idx
@torch.no_grad
def eval():
    bm1.eval()
    out={}
    for split in ['train', 'val']:
      losses=torch.zeros(eval_iters)
      for k in range(eval_iters):
         X, Y=get_batch(split)
         logits, loss= bm1(X, Y)
         losses[k]=loss
      out[split]=losses.mean()
    return out
    bm1.train()












bm1=BigramModel()
bm1=bm1.to(device)

optimizer=torch.optim.AdamW(bm1.parameters(), lr)


for _ in range(max_iters):
  xb, yb=get_batch("train")
  logits, loss=bm1(xb, yb)
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()
  if _ % eval_inter==0:
     losses=eval()
     print(f"step:{_} train loss:{losses['train']}, val loss:{losses['val']}")
  if _==4999:
    losses=eval()
    print(f"step:{_} train loss:{losses['train']}, val loss:{losses['val']}")




step:0 train loss:3.739708185195923, val loss:3.7360925674438477
step:500 train loss:1.4468269348144531, val loss:1.4616193771362305
step:1000 train loss:1.1059880256652832, val loss:1.1338573694229126
step:1500 train loss:0.9873049855232239, val loss:1.0126242637634277
step:2000 train loss:0.9150543808937073, val loss:0.9438263773918152
step:2500 train loss:0.875654935836792, val loss:0.9028713703155518
step:3000 train loss:0.8363196849822998, val loss:0.8693223595619202
step:3500 train loss:0.8099381923675537, val loss:0.83941650390625
step:4000 train loss:0.7890445590019226, val loss:0.8213627338409424
step:4500 train loss:0.7656910419464111, val loss:0.7986171245574951
step:4999 train loss:0.746448814868927, val loss:0.7833225727081299


In [10]:
n_of_tokens=10000 #no of tokens to generate
print(decode(bm1.generate(torch.zeros((1, 1), dtype=torch.long, device=device), n_of_tokens)[0].tolist()))


Tommy cried over to the birds, house was very mighty. He has a car and a bell. He wanted to share his house. He opened his owner and told his friends to eat.
So, they added each other's peaceful search. The man bit his hand. But he needed it safe again. He missed his problems and peaceful. As they did, they moved away, dancing in front of his own peace. He was so grateful that he could help the buttons over. He saiked, "Don't worry, I don't come back.
<|endoftext|>
 
Once there was a little girl who loved like her smooth. She always smiled at her mommy sound and let her patch her.
One day, the little girl was getting closer, and the girl jumped outfits of the door to the ground. She tried to grumble to do it, but that they were bunny. She was faster and grateful, so it passed TV.
They had so much fun that they could go home. After they had, the polm decided to kay. 
The little girl was so happy! She was very proud and she spraised the baby movies. 
She put on her new toy crys, but the

In [ ]:
PATH='/content/drive/MyDrive/tinygpttinystories.pt'
torch.save(bm1.state_dict(), PATH)